## Lesson 3~4. Baseball Players, Aadhaar Data, Github API
### - Data Wrangling (CSV, SQL, API)

In [84]:
import pandas as pd
import numpy as np

def add_full_name(path_to_csv, path_to_new_csv):
    '''
    concatenate first and last names of baseball players
    '''
    baseball_df = pd.read_csv(path_to_csv)    
    baseball_df['nameFull'] = baseball_df['nameFirst'] + " " + baseball_df['nameLast']
    
    baseball_df.to_csv(path_to_new_csv)
    
def imputation(path_to_csv, path_to_new_csv):
    '''
    handle missing data with imputation
    '''
    baseball_df = pd.read_csv(path_to_csv)
    baseball_df['weight'] = baseball_df['weight'].fillna(np.mean(baseball_df['weight']))
    
    baseball_df.to_csv(path_to_new_csv)
    
add_full_name('./Data/Master.csv', './Data/Master.csv')
imputation('./Data/Master.csv', './Data/Master.csv')

In [85]:
import pandasql as pdsql

def select_first_50(filepath):
    '''
    query the first 50 observations
    '''
    aadhaar_data = pd.read_csv(filepath)
    aadhaar_data.rename(columns = lambda x : x.replace(" ", "_").lower(), inplace = True)
    
    q = """Select registrar, enrolment_agency From aadhaar_data Limit 50;"""
    
    first_50 = pdsql.sqldf(q.lower(), locals())
    
    return first_50

def aggregate_query(filepath):
    '''
    sum values by grouping
    '''
    aadhaar_data = pd.read_csv(filepath)
    aadhaar_data.rename(columns = lambda x : x.replace(" ", "_").lower(), inplace = True)
    
    q = """Select district, gender, sum(aadhaar_generated) From aadhaar_data
    Where age > 50 Group By district, gender Limit 50;"""
    
    aggr_query = pdsql.sqldf(q.lower(), locals())
    
    return aggr_query

print("First 50 rows of aadhaar data")
print(select_first_50('./Data/aadhaar_data.csv'))
print("\nUser of addhaar by district and gender")
print(aggregate_query('./Data/aadhaar_data.csv'))

First 50 rows of aadhaar data
         registrar             enrolment_agency
0   Allahabad Bank            Tera Software Ltd
1   Allahabad Bank            Tera Software Ltd
2   Allahabad Bank  Vakrangee Softwares Limited
3   Allahabad Bank  Vakrangee Softwares Limited
4   Allahabad Bank  Vakrangee Softwares Limited
5   Allahabad Bank  Vakrangee Softwares Limited
6   Allahabad Bank  Vakrangee Softwares Limited
7   Allahabad Bank  Vakrangee Softwares Limited
8   Allahabad Bank  Vakrangee Softwares Limited
9   Allahabad Bank  Vakrangee Softwares Limited
10  Allahabad Bank  Vakrangee Softwares Limited
11  Allahabad Bank  Vakrangee Softwares Limited
12  Allahabad Bank  Vakrangee Softwares Limited
13  Allahabad Bank  Vakrangee Softwares Limited
14  Allahabad Bank  Vakrangee Softwares Limited
15  Allahabad Bank  Vakrangee Softwares Limited
16  Allahabad Bank  Vakrangee Softwares Limited
17  Allahabad Bank  Vakrangee Softwares Limited
18  Allahabad Bank  Vakrangee Softwares Limited
19  Allaha

In [86]:
import json
import requests

url = 'https://api.github.com/repos/hadley/ggplot2/commits'

def api_get_request(url):
    '''
    access a certain key value by navigating json structure
    '''
    data = requests.get(url).text
    data = json.loads(data)
    
    print(data[0]['commit']['author']['email'])

api_get_request(url)

wilke@austin.utexas.edu


### - Problem Set. NYC Subway Weather

In [87]:
# quiz 1 - number of days it rained

filepath = './Data/weather-underground.csv' # for quiz 1~4

def num_rainy_day(filepath):
    weather_df = pd.read_csv(filepath)
    
    q = """Select sum(rain) From weather_df;"""
    
    rainy_day = pdsql.sqldf(q.lower(), locals())
    
    print(rainy_day)
    
num_rainy_day(filepath)

   sum(rain)
0         10


In [88]:
# quiz 2 - maximum temperature on a foggy/non-foggy day

def max_temp_aggr_by_fog(filepath):
    weather_df = pd.read_csv(filepath)
    
    q = """Select fog, max(maxtempi) From weather_df Group By fog;"""
    
    maxtemp = pdsql.sqldf(q.lower(), locals())
    
    print(maxtemp)
    
max_temp_aggr_by_fog(filepath)

   fog  max(maxtempi)
0    0             86
1    1             81


In [89]:
# quiz 3 - average of mean temperature on weekends

def avg_weekend_temperature(filepath):
    weather_df = pd.read_csv(filepath)
    
    q = """Select avg(meantempi) From weather_df 
    Where cast(strftime('%w', date) as integer) in (0, 6);"""
    
    avg_temp = pdsql.sqldf(q.lower(), locals())
    
    print(avg_temp)
    
avg_weekend_temperature(filepath)

   avg(meantempi)
0       65.111111


In [90]:
# quiz 4 - average of minimum temperature on rainy days

def avg_min_temperature(filepath):
    weather_df = pd.read_csv(filepath)
    
    q = """Select avg(mintempi) From weather_df
    Where rain = 1 And mintempi > 55"""
    
    avg_mintemp = pdsql.sqldf(q.lower(), locals())
    
    print(avg_mintemp)
    
avg_min_temperature(filepath)

   avg(mintempi)
0          61.25


In [91]:
# quiz 5 - read and manipulate csv file

import csv

filename = 'turnstile_110528.txt'

f_in = open('./Data/' + filename, 'r')
f_out = open('./Data/' + 'updated_' + filename, 'w')

reader_in = csv.reader(f_in, delimiter = ',')
writer_out = csv.writer(f_out, delimiter = ',')

for line in reader_in:
    header = line[:3]
    updated = []
    count = 0
    
    for word in line[3:]:
        updated.append(word)
        count += 1
        
        if count == 5:
            row = header + updated
            writer_out.writerow(row)
            updated = []
            count = 0

f_in.close()            
f_out.close()

In [137]:
# quiz 6 - combine files and attach the column header

filenames = ['updated_turnstile_110528.txt', 'another_turnstile_110528.txt']

with open('./Data/' + 'combined_turnstile.txt', 'w') as master_file:
    master_file.write('C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn\n')
    for filename in filenames:
        with open('./Data/' + filename) as infile:
            master_file.write(infile.read())

In [138]:
# quiz 7 - filter the data on the given condition

def filter_by_regular(filename):
    turnstile_df = pd.read_csv(filename)
    turnstile_df = turnstile_df[turnstile_df['DESCn'] == 'REGULAR']
    
    return turnstile_df

filter_by_regular('./Data/combined_turnstile.txt').to_csv('./Data/turnstile_regular.txt', index = False)

In [139]:
# quiz 8 - extract hourly entry data

def get_hourly_entries(df):
    df['ENTRIESn_hourly'] = df['ENTRIESn'] - df['ENTRIESn'].shift(1)
    df['ENTRIESn_hourly'].fillna(1, inplace = True)
    df['ENTRIESn_hourly'] = df['ENTRIESn_hourly'].astype(int)
    
    return df

turnstile_df = pd.read_csv('./Data/turnstile_regular.txt')
get_hourly_entries(turnstile_df).to_csv('./Data/turnstile_hourly_entry.txt', index = False)
turnstile_df.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly
0,A002,R051,02-00-00,05-21-11,00:00:00,REGULAR,3169391,1097585,1
1,A002,R051,02-00-00,05-21-11,04:00:00,REGULAR,3169415,1097588,24
2,A002,R051,02-00-00,05-21-11,08:00:00,REGULAR,3169431,1097607,16
3,A002,R051,02-00-00,05-21-11,12:00:00,REGULAR,3169506,1097686,75
4,A002,R051,02-00-00,05-21-11,16:00:00,REGULAR,3169693,1097734,187


In [140]:
# quiz 9 - extract hourly exit data

def get_hourly_exits(df):
    df['EXITSn_hourly'] = df['EXITSn'] - df['EXITSn'].shift(1)
    df['EXITSn_hourly'].fillna(0, inplace = True)
    df['EXITSn_hourly'] = df['EXITSn_hourly'].astype(int)
    
    return df

turnstile_df = pd.read_csv('./Data/turnstile_hourly_entry.txt')
get_hourly_exits(turnstile_df).to_csv('./Data/turnstile_hourly_exit.txt', index = False)
turnstile_df.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly
0,A002,R051,02-00-00,05-21-11,00:00:00,REGULAR,3169391,1097585,1,0
1,A002,R051,02-00-00,05-21-11,04:00:00,REGULAR,3169415,1097588,24,3
2,A002,R051,02-00-00,05-21-11,08:00:00,REGULAR,3169431,1097607,16,19
3,A002,R051,02-00-00,05-21-11,12:00:00,REGULAR,3169506,1097686,75,79
4,A002,R051,02-00-00,05-21-11,16:00:00,REGULAR,3169693,1097734,187,48


In [144]:
# quiz 10 - extract hour in the given format

def time_to_hour(time):
    hour = int(time[0].replace("0", "") + time[1])
    
    return hour
    
turnstile_df = pd.read_csv('./Data/turnstile_hourly_exit.txt')
turnstile_df['hour'] = turnstile_df['TIMEn'].apply(lambda h : time_to_hour(h))
turnstile_df.to_csv('./Data/turnstile_hourly_complete.txt', index = False)
turnstile_df.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly,hour
0,A002,R051,02-00-00,05-21-11,00:00:00,REGULAR,3169391,1097585,1,0,0
1,A002,R051,02-00-00,05-21-11,04:00:00,REGULAR,3169415,1097588,24,3,4
2,A002,R051,02-00-00,05-21-11,08:00:00,REGULAR,3169431,1097607,16,19,8
3,A002,R051,02-00-00,05-21-11,12:00:00,REGULAR,3169506,1097686,75,79,12
4,A002,R051,02-00-00,05-21-11,16:00:00,REGULAR,3169693,1097734,187,48,16


In [155]:
# quiz 11 - convert date format

import datetime

def reformat_subway_dates(date):
    date_formatted = datetime.datetime.strptime(date, "%m-%d-%y").strftime("%Y-%m-%d")
    
    return date_formatted

turnstile_df = pd.read_csv('./Data/turnstile_hourly_complete.txt')
turnstile_df['DATEn'] = turnstile_df['DATEn'].apply(lambda d : reformat_subway_dates(d))
turnstile_df.to_csv('./Data/turnstile_final.txt', index = False)
turnstile_df.head()

,C/A,UNIT,SCP,DATEn,TIMEn,DESCn,ENTRIESn,EXITSn,ENTRIESn_hourly,EXITSn_hourly,hour
0,A002,R051,02-00-00,2011-05-21,00:00:00,REGULAR,3169391,1097585,1,0,0
1,A002,R051,02-00-00,2011-05-21,04:00:00,REGULAR,3169415,1097588,24,3,4
2,A002,R051,02-00-00,2011-05-21,08:00:00,REGULAR,3169431,1097607,16,19,8
3,A002,R051,02-00-00,2011-05-21,12:00:00,REGULAR,3169506,1097686,75,79,12
4,A002,R051,02-00-00,2011-05-21,16:00:00,REGULAR,3169693,1097734,187,48,16
